感谢：
BV1Ab421J7jY

In [1]:
import torch
fp16_1 = torch.tensor(9.2300, dtype=torch.float16).cuda()
print(fp16_1)
fp16_2 = torch.tensor(9.2301, dtype=torch.float16).cuda()
print(fp16_2)


fp32_1 = torch.tensor(9.2300, dtype=torch.float32).cuda()
print(fp32_1)
fp32_2 = torch.tensor(9.2301, dtype=torch.float32).cuda()
print(fp32_2)


tensor(9.2266, device='cuda:0', dtype=torch.float16)
tensor(9.2266, device='cuda:0', dtype=torch.float16)
tensor(9.2300, device='cuda:0')
tensor(9.2301, device='cuda:0')


In [ ]:
#上溢出
fp16_up = torch.tensor(2**15+1023*2**5, dtype=torch.float16).cuda()
print(fp16_up)

fp16_up_over_1 = torch.tensor(65505, dtype=torch.float16).cuda()
print(fp16_up_over_1)

fp16_up_over_2 = torch.tensor(65520, dtype=torch.float16).cuda()
print(fp16_up_over_2)

'''
65520 是float16 从65504跳到inf的临界值,具体推导看md文件
'''


tensor(65504., device='cuda:0', dtype=torch.float16)
tensor(65504., device='cuda:0', dtype=torch.float16)
tensor(inf, device='cuda:0', dtype=torch.float16)


In [ ]:
# 下溢出

'''
当指数位为0时,计算公式发生变化
公式为 2**(-14)*(0+x/1024)
临界值在x为0.5时，这里不是四舍五入，而是x取0.5时，就直接舍去为0
'''
fp16_down = torch.tensor(2**(-24)*0.501, dtype=torch.float16).cuda()
print(fp16_down)

fp16_down_over_1 = torch.tensor(2**(-25), dtype=torch.float16).cuda()
print(fp16_down_over_1)


tensor(5.9605e-08, device='cuda:0', dtype=torch.float16)
tensor(0., device='cuda:0', dtype=torch.float16)


In [12]:
#Nan
## softmax 函数
m1=torch.tensor(19990, dtype=torch.float16).cuda()
m2=torch.tensor(20000, dtype=torch.float16).cuda()
print(torch.exp(m1)/(torch.exp(m1)+torch.exp(m2)))
print(m1,m2)
## 解决Nan

print(torch.exp(m1-max(m1,m2))/(torch.exp(m1-max(m1,m2))+torch.exp(m2-max(m1,m2))))



tensor(nan, device='cuda:0', dtype=torch.float16)
tensor(19984., device='cuda:0', dtype=torch.float16) tensor(20000., device='cuda:0', dtype=torch.float16)
tensor(1.1921e-07, device='cuda:0', dtype=torch.float16)


In [ ]:
# Nan2
s1=torch.tensor(50*[1000], dtype=torch.float16).cuda()
print(torch.mean(s1))

# 这个在torch 2.3.1 里不会有问题，已经优化了
# 我估摸着低版本torch会有问题
# 这里的问题是指，batchsize过大，会优先相加分子，导致过大溢出。
# 按理说大于 65504就会变成inf,但是torch的实现是先相加分子，然后再除以分母
# 所以猜测这里用了迭代法计算均值
# 但似乎torch调用的是cpp的底层mean实现，所以也许与cpp版本有关
s2=torch.tensor(500000*[1000], dtype=torch.float16).cuda()
print(torch.mean(s2))

tensor(1000., device='cuda:0', dtype=torch.float16)
tensor(1000., device='cuda:0', dtype=torch.float16)


In [ ]:
## 数值精度

import torch

pi_16=torch.tensor(3.141, dtype=torch.float16)
pi_32=torch.tensor(3.141, dtype=torch.float32)

print(pi_16)
print(pi_32)
# 3.141 float16转float32表示
print(pi_16.float())


s=torch.tensor(0.0005, dtype=torch.float16)
print(s+pi_16) # 加了等于没加
print(s+pi_32)
print(s+pi_16.float())

tensor(3.1406, dtype=torch.float16)
tensor(3.1410)
tensor(3.1406)
tensor(3.1406, dtype=torch.float16)
tensor(3.1415)
tensor(3.1411)
